In [1]:
pip install datasets scikit-learn

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\programdata\anaconda3\envs\riset\lib\site-packages\paho_mqtt-1.6.1-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at c:\programdata\anaconda3\envs\riset\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import re
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer  
stop_words = stopwords.words('english')


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report

import matplotlib.pyplot as plt
import seaborn as sns

from datasets import load_dataset
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

In [3]:
# Try loading a specific split of the dataset
dataset = load_dataset("google-research-datasets/go_emotions")

# Check the loaded data
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 43410
    })
    validation: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 5426
    })
    test: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 5427
    })
})


In [4]:
# Memeriksa fitur dari dataset
print(dataset['train'].features)

# Menampilkan semua label yang ada dalam dataset
labels = dataset['train'].features['labels']
print("Label IDs:", labels)

# Jika label berupa ID, maka tampilkan nama label dari IDs tersebut
if isinstance(labels, dict) and 'feature' in labels:
    label_names = labels['feature'].names
    print("Label names:", label_names)
else:
    print("Labels tidak memiliki atribut 'names'")

{'text': Value(dtype='string', id=None), 'labels': Sequence(feature=ClassLabel(names=['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral'], id=None), length=-1, id=None), 'id': Value(dtype='string', id=None)}
Label IDs: Sequence(feature=ClassLabel(names=['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral'], id=None), length=-1, id=None)
Labels tidak memiliki atribut 'names'


In [7]:
# Combine the training and validation data
data = dataset['train']
texts = data['text']
labels = data['labels']

# TF-IDF vectorizer
max_features = 10000  # Number of unique words to use
tfidf_vectorizer = TfidfVectorizer(max_features=max_features)
x_data = tfidf_vectorizer.fit_transform(texts).toarray()

# Convert labels to binary matrix
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y_data = mlb.fit_transform(labels)

# K-Fold Cross Validation
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

from sklearn.metrics import multilabel_confusion_matrix

kf = KFold(n_splits=5, shuffle=True, random_state=42)
accuracies = []

for train_index, val_index in kf.split(x_data):
    x_train, x_val = x_data[train_index], x_data[val_index]
    y_train, y_val = y_data[train_index], y_data[val_index]
    
    # Create a Random Forest model
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(x_train, y_train)
    
    # Predict and evaluate
    y_pred = model.predict(x_val)
    accuracy = accuracy_score(y_val, y_pred)
    accuracies.append(accuracy)
    print(f'Fold accuracy: {accuracy:.4f}')
    
    # Classification report and confusion matrix for the last validation fold
    if train_index.shape[0] == x_data.shape[0] - val_index.shape[0]:
        print("Classification Report:")
        print(classification_report(y_val, y_pred))
        print("Confusion Matrix:")
        print(multilabel_confusion_matrix(y_val, y_pred))

# Calculate the mean accuracy
print(f'Mean accuracy: {np.mean(accuracies):.4f}')

Fold accuracy: 0.3075
Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.29      0.43       863
           1       0.82      0.47      0.60       453
           2       0.73      0.11      0.19       323
           3       0.38      0.01      0.02       483
           4       0.70      0.03      0.06       577
           5       0.67      0.03      0.05       212
           6       0.71      0.02      0.04       258
           7       0.90      0.04      0.08       460
           8       0.73      0.12      0.21       128
           9       0.00      0.00      0.00       244
          10       1.00      0.01      0.01       383
          11       1.00      0.13      0.23       156
          12       1.00      0.02      0.03        58
          13       0.47      0.04      0.07       175
          14       0.86      0.05      0.10       116
          15       0.98      0.76      0.86       544
          16       0.00      0.00   

C:\ProgramData\anaconda3\envs\riset\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\envs\riset\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold accuracy: 0.3006
Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.31      0.45       815
           1       0.83      0.41      0.55       469
           2       0.54      0.09      0.15       304
           3       0.33      0.01      0.01       495
           4       0.85      0.04      0.07       618
           5       0.60      0.03      0.05       210
           6       1.00      0.01      0.01       284
           7       0.76      0.03      0.06       447
           8       0.78      0.11      0.19       132
           9       0.33      0.00      0.01       261
          10       0.62      0.01      0.02       412
          11       0.75      0.08      0.14       150
          12       1.00      0.02      0.03        63
          13       0.58      0.04      0.08       172
          14       0.71      0.04      0.07       136
          15       0.98      0.80      0.88       549
          16       0.00      0.00   

C:\ProgramData\anaconda3\envs\riset\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\envs\riset\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold accuracy: 0.3040
Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.31      0.45       829
           1       0.82      0.50      0.62       467
           2       0.71      0.12      0.21       321
           3       0.33      0.00      0.00       515
           4       0.68      0.03      0.05       605
           5       0.33      0.01      0.02       211
           6       0.60      0.01      0.02       277
           7       0.86      0.04      0.08       430
           8       0.75      0.16      0.27       130
           9       0.50      0.00      0.01       244
          10       0.50      0.01      0.02       385
          11       0.90      0.11      0.19       168
          12       1.00      0.03      0.07        59
          13       0.35      0.03      0.06       178
          14       0.60      0.05      0.10       116
          15       0.95      0.79      0.86       539
          16       0.00      0.00   

C:\ProgramData\anaconda3\envs\riset\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\envs\riset\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold accuracy: 0.3011
Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.28      0.42       832
           1       0.83      0.45      0.58       455
           2       0.67      0.12      0.20       303
           3       0.67      0.01      0.02       518
           4       0.81      0.04      0.08       594
           5       1.00      0.03      0.05       231
           6       0.60      0.01      0.02       272
           7       0.76      0.03      0.06       410
           8       0.80      0.12      0.21       131
           9       1.00      0.00      0.01       262
          10       0.67      0.01      0.02       433
          11       1.00      0.06      0.11       158
          12       1.00      0.02      0.03        65
          13       0.55      0.04      0.07       160
          14       1.00      0.08      0.14       104
          15       0.96      0.82      0.89       515
          16       0.00      0.00   

C:\ProgramData\anaconda3\envs\riset\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\envs\riset\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold accuracy: 0.3021
Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.29      0.43       791
           1       0.79      0.48      0.60       484
           2       0.74      0.10      0.18       316
           3       0.40      0.00      0.01       459
           4       0.64      0.03      0.05       545
           5       0.75      0.03      0.05       223
           6       0.60      0.02      0.04       277
           7       0.77      0.04      0.07       444
           8       0.77      0.14      0.24       120
           9       0.57      0.02      0.03       258
          10       0.20      0.00      0.00       409
          11       0.86      0.11      0.20       161
          12       1.00      0.03      0.07        58
          13       0.64      0.05      0.10       168
          14       0.86      0.05      0.09       124
          15       0.94      0.77      0.85       515
          16       0.00      0.00   

C:\ProgramData\anaconda3\envs\riset\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\envs\riset\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
